In [27]:
import sys
if '..' not in sys.path:
    sys.path.append('..')
import itertools
import pandas as pd
import numpy as np

from pystruct.learners import OneSlackSSVM
from pystruct.models import MultiLabelClf

import dataset

In [7]:
pred_df = pd.read_csv('../runs/resnet50-aug/valid.csv', index_col=0)
pred_df = pred_df.groupby(level=0).mean()
pred_data = pred_df.as_matrix()

train_df = pd.read_csv('../data/train_v2.csv', index_col=0)
train_data = np.zeros((len(pred_df), dataset.N_CLASSES), dtype=np.uint8)
for i, tags in enumerate(train_df.loc[pred_df.index]['tags']):
    for tag in tags.split():
        train_data[i, dataset.CLASSES.index(tag)] = 1

In [8]:
train_data.shape, pred_data.shape

((8096, 17), (8096, 17))

In [28]:
n_labels = dataset.N_CLASSES
full = np.vstack([x for x in itertools.combinations(range(n_labels), 2)])
model = MultiLabelClf(edges=full)
learner = OneSlackSSVM(model, show_loss_every=1, verbose=1)
learner.fit(pred_data, train_data)

Training 1-slack dual structural SVM
iteration 0
current loss: 0.168914
cutting plane objective: 0.000112, primal objective 137632.000000
iteration 1
new constraint too weak.
no additional constraints
calls to inference: 24288
final primal objective: 14199.576713 gap: 14199.576600


OneSlackSSVM(C=1.0, break_on_bad=False, cache_tol='auto',
       check_constraints=False, inactive_threshold=1e-05,
       inactive_window=50, inference_cache=0, logger=None, max_iter=10000,
       model=MultiLabelClf(n_states: 2, inference_method: ad3), n_jobs=1,
       negativity_constraint=None, show_loss_every=1, switch_to=None,
       tol=0.001, verbose=1)

In [29]:
learner.predict(pred_data[:10])

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]

In [13]:
train_data[:10]

array([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]], dtype=uint8)

In [14]:
from pystruct.datasets import load_scene
scene = load_scene()

In [16]:
for k, v in sorted(scene.items()):
    print(k, v.shape)

X_test (1196, 294)
X_train (1211, 294)
y_test (1196, 6)
y_train (1211, 6)


In [20]:
scene['X_train']

array([[ 0.646467 ,  0.666435 ,  0.685047 , ...,  0.247298 ,  0.0140249,
         0.0297093],
       [ 0.770156 ,  0.767255 ,  0.761053 , ...,  0.137833 ,  0.0826722,
         0.0363203],
       [ 0.793984 ,  0.772096 ,  0.76182  , ...,  0.0511252,  0.112506 ,
         0.0839236],
       ..., 
       [ 0.85639  ,  1.       ,  1.       , ...,  0.0194639,  0.0221667,
         0.0437381],
       [ 0.805592 ,  0.80417  ,  0.811438 , ...,  0.346736 ,  0.231481 ,
         0.332623 ],
       [ 0.855064 ,  0.858896 ,  0.911177 , ...,  0.262119 ,  0.104471 ,
         0.34728  ]])

In [21]:
scene['y_train']

array([[1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       ..., 
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])